In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import re
import os
from scipy.stats import norm
import scipy.stats

In [2]:
here = os.getcwd()
data = here + '/data'

In [3]:
# Get the treated data from project manager

In [4]:
a = pd.read_csv(data + '/rc.csv', header=0,)

In [5]:
# Get the existing location information
# this includes city, post code, river or lake etc...
# retrieved from current db for app
# or in the case of hdrc do ti by hand

In [6]:
c_creek = {'location':'cordova_creek',
           'latitude':-121.317421,
           'longitude':38.601989,
           'city':'Rancho Cordova',
           'post':95670,
           'water':'river',
           'project':'hdc'}

In [7]:
# get the codes
codes = pickle.load( open(data + '/codes.p',"rb" ) )

In [8]:
# compare the location names
# see if there were any added by the pm since the last upload

In [9]:
a['Location'].value_counts()

Cordova Creek       56
Cordova Creek II     1
Antelope             1
Arcade Creek         1
Name: Location, dtype: int64

In [10]:
# okay there were some additions so we need to construct those dicts
arcade = {'location':'arcade_creek',
           'latitude':-121.337193,
           'longitude':38.656328,
           'city':'Sacramento',
           'post':95864,
           'water':'river',
           'project':'hdc'}

In [11]:
# okay there were some additions so we need to construct those dicts
antelope = {'location':'antelope',
           'latitude':-121.366282,
           'longitude':38.713709,
           'city':'Sacramento',
           'post':95864,
           'water':'river',
           'project':'hdc'}

In [12]:
c_creek2 = {'location':'cordova_creek2',
           'latitude':-121.309888,
           'longitude':38.606566,
           'city':'Rancho Cordova',
           'post':95670,
           'water':'river',
           'project':'hdc'}

In [13]:
# hdrc will have its own datatables
# so get this in csv format so it can be uploaded to the database
loc_list = [c_creek2, antelope, arcade, c_creek]
locs = pd.DataFrame(loc_list)
locs.set_index('location', inplace=True)
locs.to_csv(data + '/hdrc_beaches.csv')

In [14]:
# okay now reduce the slr_final data to what we need
# take a look at the columns
a_cs = a.columns

In [15]:
a_cs[:20]

Index(['Location', 'Date', 'length', 'G1', 'G10', 'G100', 'G101', 'G102',
       'G11', 'G12', 'G124', 'G125', 'G126', 'G128', 'G13', 'G130', 'G131',
       'G139', 'G14', 'G140'],
      dtype='object')

In [16]:
a.rename(columns={'Location':'location', 'Date':'date', 'Total':'quantity'}, inplace=True)

In [17]:
# check the date format
type(a['date'][1])

str

In [18]:
# take a look
a['date'][:1]

0    2017-05-17
Name: date, dtype: object

In [19]:
# okay that should work, that means we can use a common library to turn these strings inot to time stamps
pd.to_datetime(a['date'][:1])

0   2017-05-17
Name: date, dtype: datetime64[ns]

In [20]:
code_list = ['G3','G4','G7','G8','G10','G13','G21','G22','G23',
             'G24','G25','G26','G27','G28','G30','G31','G32','G33','G34','G35','G37','G49','G50','G52',
           'G59','G65','G66','G67','G70','G71','G79','G80','G82','G83','G87','G89','G90','G91','G93',
           'G95','G96','G99','G100','G101','G124','G125','G126','G128','G131','G133','G134','G137','G138',
           'G142','G144','G145','G147','G151','G152','G153','G154','G155','G156','G158','G159','G160','G165',
           'G167','G171','G172','G175','G176','G177','G178','G179','G182','G188','G191','G194','G195','G198',
           'G199','G200','G203','G204','G208','G210','G211','G213',]

In [21]:
def make_item_counts_pcm():
    d = []
    for e, b in a.iterrows():
        for code in code_list:
            if code in list(a_cs):
                if b[code] > 0:
                    c = {'location':b['location'],'date':b['date'], 'length':b['length'], 'quantity':b[code], 'density': b[code]/b['length'], 'code':code}
                d.append(c)
    return d
hdrc = make_item_counts_pcm()


In [22]:
# take a look
hdrc[:1]

[{'location': 'Antelope',
  'date': '2017-05-17',
  'length': 60.96,
  'quantity': 29,
  'density': 0.47572178477690286,
  'code': 'G3'}]

In [26]:
# that works with no errors
hdrc_pcs = pd.DataFrame(hdrc)


In [27]:
# take a look
hdrc_pcs[:1]

,code,date,density,length,location,quantity
0,G3,2017-05-17,0.475722,60.96,Antelope,29


In [29]:
hdrc_pcs.set_index('location', inplace=True)
hdrc_pcs['project'] = 'hdc'

In [30]:
hdrc_pcs = hdrc_pcs[['date', 'length', 'quantity', 'code', 'project']].copy()

In [31]:
# take a look
hdrc_pcs[:1]

,date,length,quantity,code,project
location,,,,,
Antelope,2017-05-17,60.96,29,G3,hdc


In [32]:
hdrc_pcs.to_csv(data + '/hdrc_pcs.csv')

In [ ]:
new_cols =['BeachName','BeachLength_m','EventDate','G3','G4','G7','G8','G10','G13','G21','G22','G23',
           'G24','G25','G26','G27','G28','G30','G31','G32','G33','G34','G35','G37','G49','G50','G52',
           'G59','G65','G66','G67','G70','G71','G79','G80','G82','G83','G87','G89','G90','G91','G93',
           'G95','G96','G99','G100','G101','G124','G125','G126','G128','G131','G133','G134','G137','G138',
           'G142','G144','G145','G147','G151','G152','G153','G154','G155','G156','G158','G159','G160','G165',
           'G167','G171','G172','G175','G176','G177','G178','G179','G182','G188','G191','G194','G195','G198',
           'G199','G200','G203','G204','G208','G210','G211','G213','TOTAL','Area','DENSITY']

In [ ]:
a_1 = a[new_cols].copy()

In [ ]:
obs_v = a[['BeachName','Percent_WashedAshore']].copy()

In [ ]:
# Need to identify all records where area = 0
# This will give a 'undefined' value for 'DENSITY'


In [ ]:
a_1.fillna(0, inplace=True)

In [ ]:
no_area = a_1.loc[a_1.Area == 0].copy()
no_area.to_csv('data/no_area.csv')

In [ ]:
a_1.rename(columns={'BeachName':'location', 'BeachLength_m':'length', 'EventDate':'date'}, inplace=True)

In [ ]:
# okay now we need to drop the records that have an area = 0
# notice I keep the original dataframe and make a new one

In [ ]:
a_2 = a_1.loc[a_1.Area > 0].copy()

In [ ]:
def make_item_counts_area():
    d = []
    for a, b in a_2.iterrows():
        for code in code_list:
            if b[code] > 0:
                c = {'location':b['location'],'date':b['date'].strftime("%Y-%m-%d"), 'area':b['Area'], 'quantity':b[code], 'density': b[code]/b['Area'], 'code':code}
                d.append(c)
    return d
a_per_item_area = make_item_counts_area()

In [ ]:
# get the total of all the items found
# then get the daily density of all items per location per date

In [ ]:
a_2['total']=a_2[code_list].sum(axis=1)
a_1['total']=a_1[code_list].sum(axis=1)
a_1 = a_1[a_1.total > 0].copy()

In [ ]:
# the remote database has density as the column name for pcs/m
# will keep as is and name the total/area column as density2
# saves alot of work, and that can be changed later
a_1['density'] = a_1['total']/a_1['length']
a_2['density2'] = a_2['total']/a_2['Area']

In [ ]:
slr_dens_date = a_1[['location', 'date', 'density', 'total']].copy()
slr_dens_date2 = a_2[['location', 'date', 'density2', 'total']].copy()

In [ ]:
def change_name(df):
    df.rename(columns={'total':'quantity'})
change_name(slr_dens_date)

In [ ]:
change_name(slr_dens_date2)

In [ ]:
def get_sample_no(df):
    d = []
    for name in nme_p:
        a = df.loc[df.location == name]
        a = a.sort_values(by='date')
        a.reset_index(inplace=True)
        for i, row in a.iterrows():
            n=i+1
            c = {'location':row['location'], 'date':row['date'], 'sample':n, 'density2':row['density2'], 'quantity':row['total']}
            d.append(c)
    return d
slr_d_d2 = get_sample_no(slr_dens_date2)           

In [ ]:
slr_area_samps = pd.DataFrame(slr_d_d2)

In [ ]:
slr_area_samps = slr_area_samps[['location', 'date', 'sample', 'density2', 'quantity']]

In [ ]:
slr_area_samps['date'] = slr_area_samps['date'].dt.strftime("%Y-%m-%d")

In [ ]:
slr_area_samps.set_index('date', inplace=True)

In [ ]:
slr_area_samps['density2'] = slr_area_samps['density2'].round(3)
slr_area_samps['quantity'] = slr_area_samps['quantity'].astype(int)
slr_area_samps['sample'] = slr_area_samps['sample'].astype(int)

In [ ]:
slr_area_samps.to_csv('data/slr_area.csv', encoding='latin1')

In [ ]:
def get_sample_no(df):
    d = []
    for name in nme_p:
        a = df.loc[df.location == name]
        a = a.sort_values(by='date')
        a.reset_index(inplace=True)
        for i, row in a.iterrows():
            n=i+1
            c = {'location':row['location'], 'date':row['date'], 'sample':n, 'density':row['density'], 'quantity':row['total']}
            d.append(c)
    return d

In [ ]:
slr_pcs_m = get_sample_no(slr_dens_date)

In [ ]:

slr_p_m = pd.DataFrame(slr_pcs_m)
slr_p_m = slr_p_m[['location', 'date', 'sample', 'density', 'quantity']]
slr_p_m['date'] = slr_p_m['date'].dt.strftime("%Y-%m-%d")
# slr_p_m.set_index('date', inplace=True)
slr_p_m['density'] = slr_p_m['density'].round(4)
slr_p_m['quantity'] = slr_p_m['quantity'].astype(int)
slr_p_m['sample'] = slr_p_m['sample'].astype(int)


In [ ]:
slr_p_m.set_index('location', inplace=True)

In [ ]:
slr_p_m.to_csv('data/slr_pieces.csv', encoding='latin1')